<a href="https://colab.research.google.com/github/Sohaila24/GenAI-Basic-Learnings/blob/main/Neo4j_vector_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index neo4j groq sentence-transformers


In [2]:
pip install --upgrade llama-index

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from neo4j import GraphDatabase
import groq


In [9]:
pip install llama-index-embeddings-huggingface


In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


In [49]:
username = "neo4j"
password = "PvM_MzKCcuNkGnH0vrYHO2-XoIRYnUSO73AFwLa6NSY"
url = "neo4j+s://d1eb470c.databases.neo4j.io"
driver = GraphDatabase.driver(uri, auth=(username, password))


In [14]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2025-02-26 19:53:26--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.02s   

2025-02-26 19:53:26 (3.29 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [15]:
documents = SimpleDirectoryReader("./data/paul_graham").load_data()

In [16]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [43]:
groq_client = groq.Client(api_key="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")

def query_groq_llama3(prompt):
    response = groq_client.chat.completions.create(
        model="groq/llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=500
    )
    return response.choices[0].message["content"]

In [50]:
import numpy as np

def fetch_embeddings(tx):
    query = "MATCH (d:Document) RETURN d.text AS text, d.embedding AS embedding"
    return [(record["text"], record["embedding"]) for record in tx.run(query)]

with GraphDatabase.driver(url, auth=(username, password)) as driver:
    with driver.session() as session:
        documents = session.read_transaction(fetch_embeddings)

#Compute Cosine Similarity in Python
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

query_embedding = embed_model.get_text_embedding("What happened at interleaf?")
similarities = [(text, cosine_similarity(query_embedding, np.array(embedding)))
                for text, embedding in documents]

# Sort by similarity score (descending order)
similarities.sort(key=lambda x: x[1], reverse=True)

# Retrieve the most relevant document
retrieved_text = similarities[0][0]  # Top result
print(f"Retrieved Document: {retrieved_text}")


<ipython-input-50-16793100d6bc>:9: DeprecationWarning: read_transaction has been renamed to execute_read
  documents = session.read_transaction(fetch_embeddings)


Retrieved Document: What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, th

In [51]:
#Store Similarity Scores in Neo4j
def store_similarity_scores(tx, similarities):
    for text, score in similarities:
        tx.run("""
            MATCH (d:Document {text: $text})
            SET d.similarity = $score
        """, text=text, score=score)

with GraphDatabase.driver(url, auth=(username, password)) as driver:
    with driver.session() as session:
        session.write_transaction(store_similarity_scores, similarities)


<ipython-input-51-700632e48c01>:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(store_similarity_scores, similarities)


Load the Vector Index

In [57]:
from neo4j import GraphDatabase


In [59]:
# Function to fetch existing embeddings from Neo4j
def load_vector_index(tx):
    query = """
    MATCH (d:Document)
    RETURN d.text AS text, d.embedding AS embedding
    """
    return [(record["text"], record["embedding"]) for record in tx.run(query)]

#  Connect to Neo4j & retrieve stored vectors
with GraphDatabase.driver(url, auth=(username, password)) as driver:
    with driver.session() as session:
        stored_vectors = session.read_transaction(load_vector_index)

# Convert to NumPy arrays
documents = [(text, np.array(embedding)) for text, embedding in stored_vectors]

# Print first document as a test
print(f"Loaded {len(documents)} documents.")
print(f"First document: {documents[0]}")

<ipython-input-59-df90a978586b>:12: DeprecationWarning: read_transaction has been renamed to execute_read
  stored_vectors = session.read_transaction(load_vector_index)


Loaded 1 documents.
First document: ('What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to typ